In [83]:
import xarray as xr

# Load the netCDF file
dataset = xr.open_dataset('data/tasmin_W5E5v2.0_20010101-20101231.nc')

# Extract the necessary variables
lat = dataset['lat']
lon = dataset['lon']
temperature = dataset['tasmin']
time = dataset['time']

# Define the temperature threshold
threshold = 250

# Set the window size
window_size = 30

# Define the latitude and longitude bounds for India
#lat_min, lat_max = 8, 37
#lon_min, lon_max = 68, 98
lat_min, lat_max = 36, 36.5
lon_min, lon_max = 77, 77.5

In [84]:
# Filter the dataset to India and the year 2001
india_dataset = dataset.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
india_temperature = india_dataset.sel(time='2001')['tasmin']
india_temperature

<xarray.DataArray 'tasmin' (time: 365, lat: 1, lon: 1)>
[365 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 77.25
  * lat      (lat) float64 36.25
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2001-12-31
Attributes:
    standard_name:  air_temperature
    long_name:      Daily Minimum Near-Surface Air Temperature
    units:          K

In [85]:
# Create a mask where temperature is below the threshold for 30 consecutive days
mask = np.zeros_like(india_temperature, dtype=bool)
for i in range(window_size - 1, len(india_temperature)):
    if np.all(india_temperature[i - window_size + 1: i + 1] < threshold):
        mask[i] = True
len(mask)

365

In [ ]:
# Get the latitude and longitude values
lat_values = lat.values
lon_values = lon.values

# Create a meshgrid of latitude and longitude
lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)

# Flatten the latitude, longitude, and mask arrays
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()
mask_flat = mask.flatten()

# Get the valid latitude, longitude, and mask values
valid_lat = lat_flat[mask_flat]
valid_lon = lon_flat[mask_flat]

# Print the valid latitude, longitude pairs
for lat, lon in zip(valid_lat, valid_lon):
    print(f"Latitude: {lat}, Longitude: {lon}")

In [ ]:
# Filter the dataset to India and the year 2001
india_dataset = dataset.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
india_temperature = india_dataset['tasmin'].sel(time='2001')
india_temperature

In [73]:
# Create a mask where temperature is below the threshold for 30 consecutive days
rolling_mean = india_temperature.rolling(time=window_size, center=True).mean()
mask = rolling_mean < threshold

In [71]:
# Create a mask where temperature is below the threshold for 30 consecutive days
mask = india_temperature.rolling(time=window_size).mean() < threshold
mask

<xarray.DataArray 'tasmin' (time: 365, lat: 1, lon: 1)>
array([[[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

       [[False]],

...

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]],

       [[ True]]])
Coordinates:
  * lon      (lon) float64 77.25
  * lat      (lat) float64 36.25
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2001-12-31

In [30]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

# Create a new latitude and longitude grid
new_lat = np.linspace(lat_max, lat_min, mask.shape[1])
new_lon = np.linspace(lon_min, lon_max, mask.shape[2])

# Check if there are valid data points in the mask
if np.any(mask):
    # Create a map with the mask
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Plot the mask as dots
    lon_dots, lat_dots = np.meshgrid(new_lon, new_lat)
    ax.plot(lon_dots[mask[0]], lat_dots[mask[0]], 'bo', markersize=2, transform=ccrs.PlateCarree())

    # Add coastlines and gridlines
    ax.coastlines()
    ax.gridlines()

    # Set the extent of the plot
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

    # Add a title
    ax.set_title('Locations with Minimum Temperature Below 273K')

    # Show the plot
    plt.show()
else:
    print('No data points found in the mask.')

ValueError: zero-size array to reduction operation minimum which has no identity

<Figure size 1200x600 with 1 Axes>

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

# Load the netCDF file
dataset = xr.open_dataset('data/tasmin_W5E5v2.0_20010101-20101231.nc')

# Extract the necessary variables
lat = dataset['lat']
lon = dataset['lon']
temperature = dataset['tasmin']
time = dataset['time']

# Define the temperature threshold
threshold = 273

# Set the window size
window_size = 30

# Define the latitude and longitude bounds for India
lat_min, lat_max = 8, 37
lon_min, lon_max = 68, 98

# Filter the dataset to India and the year 2001
india_dataset = dataset.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
india_temperature = india_dataset['tasmin'].sel(time='2001')

# Drop NaN values from the dataset
india_temperature = india_temperature.dropna(dim='time')

# Create a mask where temperature is below the threshold for 30 consecutive days
mask = india_temperature.rolling(time=window_size).mean() < threshold

# Check if there are valid data points in the mask
valid_mask = np.all(mask, axis=0)

# Create a new latitude and longitude grid for valid data points
new_lat = lat.values[valid_mask]
new_lon = lon.values[valid_mask]

# Check if there are valid data points to plot
if np.any(valid_mask):
    # Create a map with the mask
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Plot the mask as dots
    ax.plot(new_lon, new_lat, 'bo', markersize=2, transform=ccrs.PlateCarree())

    # Add coastlines and gridlines
    ax.coastlines()
    ax.gridlines()

    # Set the extent of the plot
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

    # Add a title
    ax.set_title('Locations with Minimum Temperature Below 273K')

    # Show the plot
    plt.show()
else:
    print('No data points found in the mask.')